## **Mount Google Drive to store files & data**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


## **1) Importing Python Packages for GAN**


In [ ]:
# from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D, Dense, Conv2DTranspose
from keras.layers import Dropout
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir new_gen_images

In [ ]:
# !mkdir src_imgs
# !unzip './drive/MyDrive/paintings1k.zip' -d src_imgs


mkdir: cannot create directory ‘src_imgs’: File exists
Archive:  ./drive/MyDrive/paintings1k.zip
  inflating: src_imgs/000e1fa33ba1ffd07953c3de9898e5e1c.jpg  
  inflating: src_imgs/00afb8e719aa2ea716a5b6a54c5c55fbc.jpg  
  inflating: src_imgs/00bd05a5d525f451228196e47d51e243c.jpg  
  inflating: src_imgs/00c4203c603bf46957ad9030f6712e9ac.jpg  
  inflating: src_imgs/00c775299a9b11d6a4d310a1464d7493c.jpg  
  inflating: src_imgs/00ca56f16c0bae52185ea31f95f0484cc.jpg  
  inflating: src_imgs/00d0e9ea9c51e85b92133b019430adf8c.jpg  
  inflating: src_imgs/00d4fe0ff5de4ae04334c8c3a3a0147dc.jpg  
  inflating: src_imgs/00d643034afe01ab875b817dc5de3af5c.jpg  
  inflating: src_imgs/00dc0126ea387c24bf4a401686134308c.jpg  
  inflating: src_imgs/00e5b91aabf7e4e3ba4cad53468780fdc.jpg  
  inflating: src_imgs/00e9ac0943d8f07ddedf04408580769ac.jpg  
  inflating: src_imgs/00e9d6ab120d2758103a71ccff104a1ac.jpg  
  inflating: src_imgs/00e24bd88ccabec5dcecfbe97a9d14a7c.jpg  
  inflating: src_imgs/00edb80f98daf

In [ ]:
images_path = "./src_imgs/"

## **Resizing Data to match Neural Network Input**

In [ ]:

# import os
# # from PIL import Image
# import cv2
# reshape_size = (64,64)

# i = 0
# for image in os.listdir(images_path):
#   # print(image)

#   img = cv2.imread(os.path.join(images_path,image))
  
#   img = cv2.resize(img, reshape_size)
#   cv2.imwrite("resized_images/%d.png" % i,img)
#   # # print(img.shape)
#   i = i+1


## **2) Parameters for Neural Networks & Data**

In [ ]:
img_width = 32
img_height = 32
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(lr=0.0002)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## **3) Building Generator**





In [ ]:
def build_generator():
    model = Sequential()
    model.add(Dense(256 * 8* 8, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((8,8,256)))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    # model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    # model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
  
    model.summary()

    return model

generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16384)             1654784   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16384)             0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      524416    
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 128)      262272    
 ranspose)                                              

## **4) Building Discriminator**

In [ ]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (3,3), padding='same', ))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    model.summary()
    return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        1792      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 64, 64, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       73856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 256)      

## **5) Connecting Neural Networks to build GAN**

In [ ]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
# generator.summary()
# discriminator.summary()

## **6) Outputting Images**


In [ ]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 4, 4
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001

    # Rescale images 0 - 1
    gen_imgs = (gen_imgs + 1) / 2.0

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("currentgeneration.png")
    fig.savefig("new_gen_images/%.8f.png" % save_name)
    plt.close()

## **7) Training GAN**

In [ ]:
from PIL import Image

def train(epochs, batch_size=32, save_interval=200):

  array = []
  #PUT PATH OF RESIZED IMAGES
  path = "./resized_imgs/resized_imgs/"

  for dir in os.listdir(path):
            # print(dir)
    image = Image.open(os.path.join(path,dir))
    data = np.asarray(image)
    array.append(data)

  X_train = np.array(array)
  print(X_train.shape)

  # print(X_train.shape)
  #Rescale data between -1 and 1
  X_train = X_train / 127.5 -1.
  bat_per_epo = int(X_train.shape[0] / batch_size)
  # X_train = np.expand_dims(X_train, axis=3)
  print(X_train.shape)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    for j in range(bat_per_epo):
      #Get Random Batch
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]

      #Generate Fake Images
      noise = np.random.normal(0, 1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      #Train discriminator
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      noise = np.random.normal(0, 1, (batch_size, latent_dim))
      
      #inverse y label
      g_loss = GAN.train_on_batch(noise, valid)

    print("*******epoch: %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch,d_loss[0], 100* d_loss[1], g_loss))

      # if(epoch % save_interval) == 0:
    save_imgs(epoch)


train(100, batch_size=32, save_interval=200)

流式输出内容被截断，只能显示最后 5000 行内容。
******* 169 16 [D loss: 0.070958, acc: 96.88%] [G loss: 9.638506]
******* 169 17 [D loss: 0.026014, acc: 100.00%] [G loss: 8.868130]
******* 169 18 [D loss: 0.007764, acc: 100.00%] [G loss: 8.878654]
******* 169 19 [D loss: 0.380827, acc: 93.75%] [G loss: 9.645678]
******* 169 20 [D loss: 0.015886, acc: 100.00%] [G loss: 8.266798]
******* 169 21 [D loss: 0.081596, acc: 98.44%] [G loss: 7.928832]
******* 169 22 [D loss: 0.017617, acc: 100.00%] [G loss: 6.600283]
******* 169 23 [D loss: 0.020176, acc: 100.00%] [G loss: 7.310489]
******* 169 24 [D loss: 0.056359, acc: 95.31%] [G loss: 6.508810]
******* 169 25 [D loss: 0.029091, acc: 98.44%] [G loss: 7.329783]
******* 169 26 [D loss: 0.079817, acc: 95.31%] [G loss: 6.787994]
******* 169 27 [D loss: 0.052317, acc: 96.88%] [G loss: 6.881963]
******* 169 28 [D loss: 0.101819, acc: 95.31%] [G loss: 6.355057]
******* 169 29 [D loss: 0.013296, acc: 100.00%] [G loss: 6.761486]
******* 169 30 [D loss: 0.063013, acc: 98.4

In [ ]:
# nearlt 45mins to finish 200 epochs with batch_size =32.
import numpy as np

noise = np.random.normal(0, 1, (16, latent_dim))
gen_imgs = generator.predict(noise)
gen_imgs = (gen_imgs + 1) / 2.0
# plt.imshow(gen_imgs[2])

NameError: ignored

In [ ]:
plt.imshow(gen_imgs[6])

In [ ]:
generator.save_weights("/content/drive/MyDrive/models/generator1hour.h5")
discriminator.save_weights("/content/drive/MyDrive/models/discriminator1hour.h5")

### **8) Making GIF**

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'my_dcgan_artwork.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('new_gen_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)